In [2]:
import pandas as pd
import io
import requests
import psycopg2
from sqlalchemy import create_engine

url = 'https://raw.githubusercontent.com/krtmlry/product_sales_2019/main/data/productsales_merged_01.csv'
response = requests.get(url)
df = pd.read_csv(io.StringIO(response.text), sep=',')

# 1. Convert data types

In [3]:
df['order_id'] = pd.to_numeric(df['order_id'], errors= 'coerce').fillna(0).astype(int)
df['product'] = df['product'].astype(str)
df['qty_ordered'] = pd.to_numeric(df['qty_ordered'],  errors='coerce').fillna(0).astype(int)
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0).astype(float)
df['order_date'] = pd.to_datetime(df['order_date'],format='mixed',errors='coerce')
df['shipping_address'] = df['shipping_address'].astype(str)
df = df.sort_values('order_date').reset_index(drop=True)
df = df[['order_id','order_date','product','qty_ordered','price','shipping_address']]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185652 entries, 0 to 185651
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   order_id          185652 non-null  int64         
 1   order_date        185652 non-null  datetime64[ns]
 2   product           185652 non-null  object        
 3   qty_ordered       185652 non-null  int64         
 4   price             185652 non-null  float64       
 5   shipping_address  185652 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 8.5+ MB


# 2. Create datetime_dim to store order_date details; year,month, day, weekday

In [6]:
datetime_dim = df[['order_date']].reset_index(drop=True)
datetime_dim['datetime_dim_id'] = datetime_dim.index+1
datetime_dim['order_year'] = datetime_dim['order_date'].dt.year
datetime_dim['order_month'] = datetime_dim['order_date'].dt.month
datetime_dim['order_day'] = datetime_dim['order_date'].dt.day
datetime_dim['order_weekday'] = datetime_dim['order_date'].dt.weekday
datetime_dim = datetime_dim[['datetime_dim_id','order_date','order_year','order_month','order_day','order_weekday',]]
datetime_dim.head()

,datetime_dim_id,order_date,order_year,order_month,order_day,order_weekday
0,1,2019-01-01 03:07:00,2019,1,1,1
1,2,2019-01-01 03:40:00,2019,1,1,1
2,3,2019-01-01 04:56:00,2019,1,1,1
3,4,2019-01-01 05:53:00,2019,1,1,1
4,5,2019-01-01 06:03:00,2019,1,1,1


# 3. Extract product names and prices, create product id for each item

In [7]:
unique_products = df[['product','price']].drop_duplicates().reset_index(drop=True)
unique_products = unique_products.sort_values(by='product').reset_index(drop=True)
unique_products['product_id'] = unique_products.index+1
unique_products

,product,price,product_id
0,20in Monitor,109.99,1
1,27in 4K Gaming Monitor,389.99,2
2,27in FHD Monitor,149.99,3
3,34in Ultrawide Monitor,379.99,4
4,AA Batteries (4-pack),3.84,5
5,AAA Batteries (4-pack),2.99,6
6,Apple Airpods Headphones,150.00,7
7,Bose SoundSport Headphones,99.99,8
8,Flatscreen TV,300.00,9
9,Google Phone,600.00,10


# 3.1 Extract information from unique_products and convert into dictionary to be used for mapping

In [8]:
product_details = {}
for index, row in unique_products.iterrows():
    product_id = row['product_id']
    product_name = row['product']
    product_price = row['price']
    product_details[product_id] ={'product':product_name, 'price':product_price}
product_details

{1: {'product': '20in Monitor', 'price': 109.99},
 2: {'product': '27in 4K Gaming Monitor', 'price': 389.99},
 3: {'product': '27in FHD Monitor', 'price': 149.99},
 4: {'product': '34in Ultrawide Monitor', 'price': 379.99},
 5: {'product': 'AA Batteries (4-pack)', 'price': 3.84},
 6: {'product': 'AAA Batteries (4-pack)', 'price': 2.99},
 7: {'product': 'Apple Airpods Headphones', 'price': 150.0},
 8: {'product': 'Bose SoundSport Headphones', 'price': 99.99},
 9: {'product': 'Flatscreen TV', 'price': 300.0},
 10: {'product': 'Google Phone', 'price': 600.0},
 11: {'product': 'LG Dryer', 'price': 600.0},
 12: {'product': 'LG Washing Machine', 'price': 600.0},
 13: {'product': 'Lightning Charging Cable', 'price': 14.95},
 14: {'product': 'Macbook Pro Laptop', 'price': 1700.0},
 15: {'product': 'ThinkPad Laptop', 'price': 999.99},
 16: {'product': 'USB-C Charging Cable', 'price': 11.95},
 17: {'product': 'Vareebadd Phone', 'price': 400.0},
 18: {'product': 'Wired Headphones', 'price': 11.99

# 3.2 Separate product column to be used as dimension table

In [9]:
df_product = df[['product']]
df_product.head()

,product
0,Wired Headphones
1,USB-C Charging Cable
2,Apple Airpods Headphones
3,AAA Batteries (4-pack)
4,USB-C Charging Cable


# 3.3 Create function for product id mapping and price mapping to df_product

In [10]:
def product_id_mapping(product):
    product_id_map = {}  # Create an empty dictionary to store the mappings
    # Iterate through the items (key-value pairs) in product_details
    for key, value in product_details.items():
        product_id_map[value['product']] = key
    return product_id_map.get(product, None)  # Return None if 'product' not found

# 3.4 Create function for price mapping to df_product

In [12]:
def price_mapping(product_id):
    #use get() method
    return product_details.get(product_id).get('price')

# 3.5 Create product dimension table : product_dim

In [13]:
product_dim = df_product[['product']].reset_index(drop=True)
product_dim['product_dim_id'] = product_dim.index + 1  # surrogate key
product_dim['product_id'] = product_dim['product'].map(product_id_mapping)
product_dim['price'] = product_dim['product_id'].map(price_mapping)
product_dim = product_dim[['product_dim_id', 'product_id', 'product','price']]
product_dim.head()

,product_dim_id,product_id,product,price
0,1,18,Wired Headphones,11.99
1,2,16,USB-C Charging Cable,11.95
2,3,7,Apple Airpods Headphones,150.00
3,4,6,AAA Batteries (4-pack),2.99
4,5,16,USB-C Charging Cable,11.95


# 4. Create customer dimension table: customer_dim

In [14]:
customer_dim = df[['shipping_address']].reset_index(drop=True)
customer_dim['customer_dim_id'] = customer_dim.index+1
customer_dim['customer_id'] = customer_dim.index+1 #Natural/ main primary key
customer_dim['city'] = customer_dim['shipping_address'].str.split(', ').str[1]
customer_dim = customer_dim[['customer_dim_id','customer_id','shipping_address','city']]
customer_dim.head()

,customer_dim_id,customer_id,shipping_address,city
0,1,1,"9 Lake St, New York City, NY 10001",New York City
1,2,2,"760 Church St, San Francisco, CA 94016",San Francisco
2,3,3,"735 5th St, New York City, NY 10001",New York City
3,4,4,"75 Jackson St, Dallas, TX 75001",Dallas
4,5,5,"943 2nd St, Atlanta, GA 30301",Atlanta


# 5. Create order_details as a fact table

In [15]:
order_details = df.merge(datetime_dim, how='left', left_index=True, right_index=True) \
            .merge(customer_dim, how='left', left_index=True, right_index=True) \
            .merge(product_dim, how='left', left_index=True, right_index=True) \
            [['order_id','datetime_dim_id','customer_dim_id','product_dim_id','qty_ordered']]
order_details['total_price'] = (product_dim['price']*df['qty_ordered']).round(2)
order_details['datetime_dim_id'] = order_details['datetime_dim_id'].astype(pd.Int32Dtype())
order_details['customer_dim_id'] = order_details['customer_dim_id'].astype(pd.Int32Dtype())
order_details['product_dim_id'] = order_details['product_dim_id'].astype(pd.Int32Dtype())

In [16]:
order_details.head()

,order_id,datetime_dim_id,customer_dim_id,product_dim_id,qty_ordered,total_price
0,147268,1,1,1,1,11.99
1,148041,2,2,2,1,11.95
2,149343,3,3,3,1,150.00
3,149964,4,4,4,1,2.99
4,149350,5,5,5,2,23.90


# 6. Export tables to Supabase / Local postgres DB

In [3]:
db_creds ={
    'host': 'hostname', #replace with hostname or url
    'database': 'postgres',
    'user': 'postgres',
    'password': 'password', #replace with password
    'port': 5432}

In [31]:
conn = psycopg2.connect(**db_creds)
cur = conn.cursor()

engine = create_engine(f'postgresql://{db_creds["user"]}:{db_creds["password"]}@{db_creds["host"]}:{db_creds["port"]}/{db_creds["database"]}')

In [32]:
create_datetime_dim = '''
    CREATE TABLE datetime_dim (
    datetime_dim_id PRIMARY KEY,
    order_date TIMESTAMP,
    order_year INTEGER,
    order_month INTEGER,
    order_day INTEGER,
    order_weekday INTEGER
    )
'''

create_customer_dim = '''
    CREATE TABLE customer_dim (
    customer_dim_id PRIMARY KEY,
    customer_id INTEGER,
    purchase_address VARCHAR(250),
    city VARCHAR(250)
    );
'''

create_product_dim = '''
    CREATE TABLE product_dim (
    product_dim_id PRIMARY KEY,
    product_id INTEGER,
    product VARCHAR(250),
    price FLOAT
    );
'''

create_order_details = '''
    CREATE TABLE order_details (
    order_id PRIMARY KEY,
    datetime_dim_id INTEGER,
    customer_dim_id INTEGER,
    product_dim_id INTEGER,
    qty_ordered INTEGER,
    total_price FLOAT,
    FOREIGN KEY (datetime_dim_id) REFERENCES datetime_dim (datetime_dim_id),
    FOREIGN KEY (customer_dim_id) REFERENCES customer_dim (customer_dim_id),
    FOREIGN KEY (product_dim_id) REFERENCES product_dim (product_dim_id)
    );
'''

In [33]:
try:
    cur.execute(create_datetime_dim)
    cur.execute(create_customer_dim)
    cur.execute(create_product_dim)
    cur.execute(create_order_details)
    conn.commit()
except:
    conn.rollback()

In [34]:
datetime_dim.to_sql('datetime_dim', engine, if_exists='replace', index=False)
customer_dim.to_sql('customer_dim', engine, if_exists='replace', index=False)
product_dim.to_sql('product_dim', engine, if_exists='replace', index=False)
order_details.to_sql('order_details', engine, if_exists='replace', index=False)

# Close the database connection
conn.close()